In [147]:
import pandas as pd
import numpy as np


In [148]:
df=pd.read_csv("adult.csv",encoding='latin-1')
df[df == '?'] = np.nan
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)
X = df.drop(['income'], axis=1)
y = df['income']

In [149]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from numpy import int64



In [150]:
scores=[]

In [151]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import pickle

def logc(y_train,y_test,X_train,X_test,index):
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('Logistic Regression accuracy score with all the features: {0:0.4f}'. format(score))
    filename="logreg"+str(index)+".sav"
    pickle.dump(logreg,open(filename,"wb"))

def xgbs(y_train,y_test,X_train,X_test,index):
    bst = XGBClassifier(n_estimators=6, max_depth=8, learning_rate=0.8, objective='binary:logistic')
    bst.fit(X_train,y_train)
    y_pred = bst.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('XGBClassifier accuracy score with all the features: {0:0.4f}'. format(score))
    filename="bst"+str(index)+".sav"
    pickle.dump(bst,open(filename,"wb"))

def rfs(y_train,y_test,X_train,X_test,index):
    rf = RandomForestClassifier(random_state=0,n_estimators=18,max_depth=15)
    rf.fit(X_train,y_train)
    y_pred = rf.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('RandomForestClassifier accuracy score with all the features: {0:0.4f}'. format(score))
    filename="rf"+str(index)+".sav"
    pickle.dump(rf,open(filename,"wb"))   
def lgbs(y_train,y_test,X_train,X_test,index):
    clf = lgb.LGBMClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score=accuracy_score(y_test, y_pred)
    scores.append(score)
    print('LGBMClassifier accuracy score with all the features: {0:0.4f}'. format(score))
    filename="clf"+str(index)+".ptl"
    pickle.dump(clf,open(filename,"wb"))

In [152]:
def load_pre(filename,X_test,y_test):
    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred = loaded_model.predict(X_test)
    print(accuracy_score(y_test, y_pred))

In [153]:
def genModel(seed:int,only=False):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)
        categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
        for feature in categorical:
                le = preprocessing.LabelEncoder()
                X_train[feature] = le.fit_transform(X_train[feature])
                X_test[feature] = le.transform(X_test[feature])
        scaler = StandardScaler()

        X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

        X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

        y_train[y_train=="<=50K"]=0
        y_train[y_train==">50K"]=1
        y_test[y_test=="<=50K"]=0
        y_test[y_test==">50K"]=1
        y_train=y_train.astype(int64)
        y_test=y_test.astype(int64)
        # load_pre("/home/zyxing/adult_income/logreg0.sav",X_test=X_test,y_test=y_test)
        # lgbs(y_train=y_train,y_test=y_test,X_train=X_train,X_test=X_test,index=seed)
        if not only:
                # logc(y_train,y_test,X_train,X_test,index=seed)
                # xgbs(y_train=y_train,y_test=y_test,X_train=X_train,X_test=X_test,index=seed)
                rfs(y_train=y_train,y_test=y_test,X_train=X_train,X_test=X_test,index=seed)
        

In [154]:
for i in range(30):
    try:
        genModel(i)
    except Exception as e:
        print(e)
        continue 

RandomForestClassifier accuracy score with all the features: 0.8582
y contains previously unseen labels: 'Holand-Netherlands'
y contains previously unseen labels: 'Holand-Netherlands'
RandomForestClassifier accuracy score with all the features: 0.8616
RandomForestClassifier accuracy score with all the features: 0.8567
RandomForestClassifier accuracy score with all the features: 0.8630
y contains previously unseen labels: 'Holand-Netherlands'
RandomForestClassifier accuracy score with all the features: 0.8608
RandomForestClassifier accuracy score with all the features: 0.8609
y contains previously unseen labels: 'Holand-Netherlands'
y contains previously unseen labels: 'Holand-Netherlands'
RandomForestClassifier accuracy score with all the features: 0.8630
RandomForestClassifier accuracy score with all the features: 0.8663
y contains previously unseen labels: 'Holand-Netherlands'
RandomForestClassifier accuracy score with all the features: 0.8624
y contains previously unseen labels: 'Ho

In [155]:
np.save("scores.npy",np.array(scores))

In [156]:
# scores=[]
# for i in range(30,140):
#     try:
#         genModel(i,True)
#     except Exception as e:
#         print(e)
#         continue 
# np.save("scores.npy",np.array(scores))